In [1]:
!pip -q install -U pip setuptools wheel
!pip -q install "pycaret==3.3.2"

import pycaret
print("PyCaret version:", pycaret.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.2 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df = pd.read_csv(url, parse_dates=["Date"], index_col="Date")
df = df.asfreq("D")      # ensure daily frequency
df = df["Temp"].iloc[:365]   # keep 1 year (~365 rows)

print("Shape:", df.shape)
df.head()


Shape: (365,)


Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Temp, dtype: float64

In [4]:
from pycaret.time_series import *

# PyCaret expects a Series or a single-column DataFrame
s = setup(
    data=df,            # 1-year daily series (Temp)
    fh=14,              # forecast horizon = 14 days
    fold=3,             # small CV for speed
    session_id=42,
    use_gpu=True,       # uses GPU where supported
    verbose=False
)

best = compare_models()   # picks best forecaster by default metric (MASE/SMAPE)
best


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.8561,0.8316,1.8476,2.2786,0.1469,0.1482,-0.2375,0.0367
arima,ARIMA,0.8806,0.8749,1.9003,2.3972,0.1475,0.1536,-0.3765,0.1133
auto_arima,Auto ARIMA,0.8911,0.8836,1.9224,2.4208,0.1478,0.1559,-0.4234,1.3833
theta,Theta Forecaster,0.8977,0.8681,1.9366,2.3785,0.1547,0.1573,-0.3664,0.0400
croston,Croston,0.9004,0.8863,1.9423,2.4281,0.1505,0.1579,-0.4370,0.0200
exp_smooth,Exponential Smoothing,0.9011,0.8742,1.9440,2.3950,0.1546,0.1580,-0.3896,0.0467
ets,ETS,0.9011,0.8742,1.9440,2.3951,0.1546,0.1580,-0.3896,0.0567
grand_means,Grand Means Forecaster,0.9758,0.9474,2.1061,2.5960,0.1620,0.1711,-0.6346,0.0300
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.1256,1.1100,2.4291,3.0415,0.1771,0.2005,-1.2366,0.2933
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,1.3549,1.2740,2.9243,3.4913,0.2212,0.2576,-1.9062,5.4167


[2025-10-23 20:58:37.856] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-23 20:58:38.289] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


NaiveForecaster()

In [5]:
final_best = finalize_model(best)     # refit on full data
future_pred = predict_model(final_best)  # forecasts next fh=14 days
display(future_pred.head(20))


,y_pred
1982-01-01,17.4
1982-01-02,17.4
1982-01-03,17.4
1982-01-04,17.4
1982-01-05,17.4
1982-01-06,17.4
1982-01-07,17.4
1982-01-08,17.4
1982-01-09,17.4
1982-01-10,17.4


In [6]:
# Backtest and residual diagnostics for the final model
plot_model(final_best, plot="forecast")     # in-sample + forecast overlay
plot_model(final_best, plot="residuals")    # residual distribution
plot_model(final_best, plot="diagnostics")  # ACF/PACF etc.


In [7]:
import os
os.makedirs("/kaggle/working/media/figures", exist_ok=True)
os.makedirs("/kaggle/working/notebooks", exist_ok=True)

# Save the finalized model
save_model(final_best, "/kaggle/working/notebooks/ts_univariate_no_exog_final")

# Save the 14-day forecast
future_pred.to_csv("/kaggle/working/media/figures/ts_univariate_no_exog_forecast.csv", index=True)

print("Saved:")
print("- /kaggle/working/notebooks/ts_univariate_no_exog_final.pkl")
print("- /kaggle/working/media/figures/ts_univariate_no_exog_forecast.csv")


Transformation Pipeline and Model Successfully Saved
Saved:
- /kaggle/working/notebooks/ts_univariate_no_exog_final.pkl
- /kaggle/working/media/figures/ts_univariate_no_exog_forecast.csv
